In [1]:
import math
import torch
from diffusion_edf.dist import *
from diffusion_edf import transforms
import plotly.graph_objs as go

In [2]:
dtype = torch.float64
device = 'cpu'

eps = 1e-8
q = sample_igso3(eps = eps, N=3, dtype=dtype, device=device)
# theta = 0.1
# # q = torch.tensor([[math.cos(theta/2), math.sin(theta/2), 0., 0.]], dtype=dtype, device=device)
# randomaxis = torch.randn(1,3, dtype=dtype, device=device)
# randomaxis = randomaxis / randomaxis.norm(dim=-1, keepdim=True) * math.sin(theta/2)
# q = torch.cat([torch.ones_like(randomaxis[...,:1]) * math.cos(theta/2), randomaxis], dim=-1)

prob = igso3(q, eps)
deriv = igso3_lie_deriv(q, eps)
print(f"Anaylitic Lie-derivative: {deriv}")

dt = 0.0000001

prob_rot = []
for i in range(3):
    qrot = transforms.quaternion_multiply(q,transforms.axis_angle_to_quaternion(dt * torch.eye(3, device=q.device, dtype=q.dtype)[i]))
    prob_rot.append(igso3(qrot, eps))

prob_rot = torch.stack(prob_rot, dim=-1)
print(f"Numerical Lie-derivative: {(prob_rot - prob[...,None]) / dt}")
print(f"Allclose: {torch.allclose((prob_rot - prob[...,None]) / dt, deriv, atol=0, rtol=1e-2)}")

Anaylitic Lie-derivative: tensor([[ 1.2962e+15, -8.8996e+14, -1.3944e+15],
        [ 5.8871e+14,  2.1242e+15,  5.2753e+14],
        [ 1.1607e+15, -4.1366e+15, -1.7540e+15]], dtype=torch.float64)
Numerical Lie-derivative: tensor([[ 1.2965e+15, -8.8997e+14, -1.3939e+15],
        [ 5.8848e+14,  2.1254e+15,  5.2729e+14],
        [ 1.1567e+15, -4.1402e+15, -1.7580e+15]], dtype=torch.float64)
Allclose: True
